### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Esophageal_Cancer/GSE124158'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = False  # Based on the series summary, this dataset focuses on miRNA, not gene expression data

# Determine the availability of the 'Esophageal_Cancer' trait
trait_row = 1 if 'patient diagnosis: Esophageal Cancer' in sample_characteristics[1] else None

# Determine the availability of 'age'
age_row = 4 if len(set(sample_characteristics[4])) > 1 else None

# Determine the availability of 'gender'
gender_row = 2 if len(set(sample_characteristics[2])) > 1 else None

# Define the function to convert 'Esophageal_Cancer' values
def convert_trait(value):
    value = value.split(':')[1].strip()  # Extract the value
    if value == 'Esophageal Cancer':
        return 1
    elif value in ['Colorectal Cancer', 'Gastric Cancer', 'Glioma', 'HCC', 'Lung Cancer', 'Pancreatic Cancer', 
                   'Intermediate bone and soft tissue tumor', 'Malignant bone and soft tissue tumor', 
                   'Benign bone and soft tissue tumor', 'Healthy']:
        return 0
    else:
        return None

# Define the function to convert 'age' values
def convert_age(value):
    value = value.split(':')[1].strip()  # Extract the value
    try:
        return float(value)
    except ValueError:
        return None

# Define the function to convert 'gender' values
def convert_gender(value):
    value = value.split(':')[1].strip()  # Extract the value
    if value == 'Male':
        return 1
    elif value == 'Female':
        return 0
    else:
        return None

# Save cohort information
save_cohort_info('GSE124158', './preprocessed/Esophageal_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# If clinical data is available, extract and save the selected clinical features
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Esophageal_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Esophageal_Cancer/trait_data/GSE124158.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
